<a href="https://colab.research.google.com/github/Pudi-Bheemesh/AutismDetectorByPhoto/blob/main/Project_official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detection  of Autism Spectrum Disorder with image

This notebook is to build an end-to-end binary-class image classification using Tensorflow 2.0 and Teensorflow Hub.

## 1. Problem

Identifing weather a child has ASD(Autism Spectrum Disorder)
By  this Deep learning model we can find the symptoms of ASD and help the child with helping with hospitalisation and other methods.

## 2. Data

The data we use is from Kaggle's Autism_Image_Data dataset

https://www.kaggle.com/datasets/cihan063/autism-image-data


## 3. Evaluation

The evaluation is a file with a prediction probabilities for a child with or without autism each test page.

In [1]:
# !unzip "drive/MyDrive/data/archive.zip" -d "drive/MyDrive/data/"
# Use the '-d' parameter as the destination for where the files should go

#Getting our workspace ready

Let's run some import statements. And check whether or not we're using a GPU.

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)

# Check for GPU
print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

TF version: 2.18.0
Hub version: 0.16.1
GPU available (YESS!!!!)


a GPU is a computer chip which is faster at doing numerical computing. And since machine learning is all about finding patterns in numbers, that's what we're after.

Running this for the first time in Colab will let us know there's no GPU available.

This is because by default Colab runs on a computer located on Google's servers which doesn't have a GPU attached to it.

In [3]:
# importing necessary libraries
import os
import random
import gc

In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

##Getting data ready
Since much of machine learning is getting your data ready to be used with a machine learning model, we'll take extra care getting it setup.

There are a few ways we could do this. Many of them are detailed in the Google Colab notebook on I/O (input and output).

And because the data we're using is hosted on Kaggle, we could even use the Kaggle API.
or we can upload it to your Google Drive, mount your drive in this notebook and import the file.

This means we'll be able to access files in our Google Drive right in this notebook.

For this project, I've downloaded the data from Kaggle and uploaded it to my Google Drive as a .zip file under the folder "Data".

#Accessing the data
We create file path for training and test data from the 'autism-image-data


* We get the 'Autistic' and 'Non-Autistic' images from train data, shuffle them and use as training images
* The dataset has 2450 train images, with 1225 images for each 'Autistic' and 'Non-Autistic' category, our dataset is well balanced among the 2 classes
* We also get the 300 test images from file path

In [5]:
data_dir = "drive/MyDrive/data/AutismDataset"
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir,image_class)):
    image_path = os.path.join(data_dir,image_class,image)

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/data/AutismDataset'

In [ ]:
img = cv2.imread(os.path.join(data_dir,"train","Autistic.18.jpg"))
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
data_directory = "drive/MyDrive/data/AutismDataset/consolidated"
data = tf.keras.utils.image_dataset_from_directory(data_directory,image_size=(256,256))

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
#images represented as numpy arrays
batch[0].shape

In [ ]:
#If 0 then autistic
#If 1 then non autistic
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
scaled = batch[0] /255
scaled.max()

# 2.Preprocess the data
### Scale data
We need to scale our data to set it up as equally sized variables. We can effectively applied our step on the data pipeline

In [ ]:
data = data.map(lambda x,y: (x/255,y))

Map function maps across the elements of the dataset
this transformation applies to each element of this datasetthe transformed elements will be in the same order as the input

In [ ]:
data.as_numpy_iterator().next()[0].max()

#### Split Data

In [ ]:
train_size = int(len(data)*.6)
val_size = int(len(data)*.3)
test_size = int(len(data)*.1)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
train


# 3.Deep Model
#### 1.Building a deep learning model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
# Calling pre-trained VGG16 model
base_model = VGG16(include_top=False,weights='imagenet',input_shape=(256,256,3))
# Freeze the layers in pre-trained model, we don't need to train again
for layer in base_model.layers:
  layer.trainable = False
# Let's see how many layers are in the vgg model
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
model = Sequential()

In [ ]:
model.add(base_model)

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir = "drive/MyDrive/data/logs"

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=25, validation_data=val, callbacks=[tensorboard_callback])

#### Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

### 9. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f"Precision:{pre.result().numpy()},Recall:{re.result().numpy()},Accuracy:{acc.result().numpy()}")

### 10. Test

In [ ]:
img = cv2.imread(os.path.join(data_dir,"test","Autistic.18.jpg"))
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img,(256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
resize.shape

In [ ]:
np.expand_dims(resize,0).shape

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
    print(f'Predicted photo is Not Autistic')
else:
    print(f'Predicted photo is Autistic')

In [ ]:
data_dir

In [ ]:
img = cv2.imread(os.path.join(data_dir,"test","Non_Autistic.105.jpg"))
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img,(256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
print(resize.shape)
print(np.expand_dims(resize,0).shape)

yhat = model.predict(np.expand_dims(resize/255, 0))


In [ ]:
print(yhat)

In [ ]:
if yhat > 0.5:
    print(f'Predicted photo is Not Autistic')
else:
    print(f'Predicted photo is Autistic')

In [ ]:
img = cv2.imread(os.path.join(data_dir,"train","Non_Autistic.149.jpg"))
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
print(resize.shape)
print(np.expand_dims(resize,0).shape)

yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
print(yhat)

In [ ]:
if yhat > 0.5:
    print(f'Predicted photo is Not Autistic')
else:
    print(f'Predicted photo is Autistic')